In [1]:
import base64
import os

import cv2
import requests

path=r'C:\AIstudy\traincarmodel\train'

class_brand = ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']

url = 'http://127.0.0.1:8080/api/gethog'
def img2HOG(img):
    v,buffer = cv2.imencode('.jpg',img) 
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.post(url,json={"item_str":data})
    return response.json()

def readData(path):
    response = []
    for folder in os.listdir(path):
        for img_name in os.listdir(path+'/'+folder):
            img_path = path+'/'+folder+'/'+img_name
            img = cv2.imread(img_path)  #ได้เป็นไฟล์รูปที่เป็นตัวเลขเยอะๆ
            res = img2HOG(img)  #เรียก method  api เพื่อทำการแปลง img เป็น hog vector
            hog = list(res["HOG"]) 
            hog.append(class_brand.index(folder))   # เอาเลขไปต่อกันเฉยๆให้เป็น x , y
            response.append(hog)     #พอได้ค่าแล้วก็ทำการเก็บไว้ใน list

    return response

dataset = readData(path)
# vector คือการแปลงจากสองมิติเป็นหนึ่งมิติ เพราะเวลาที่ทำงาน ต้องมีการแปลงเป็นจากสองเป็น1 เพื่อให้เป็น vector เพื่อให้ข้อมูลมันสัมพันธ์

            

In [2]:
import pickle
write_path = "imageHogTrain.pkl"
pickle.dump(dataset, open(write_path,"wb"))

In [3]:
dataset = pickle.load(open('imageHogTrain.pkl','rb'))

In [4]:
import numpy as np
data = np.array(dataset)
print(data[1])

[0.15791625 0.17903247 0.11346089 ... 0.34124827 0.03008643 0.        ]


In [5]:
X_train = data[:,0:-1]
y_train = data[:,-1]
print(X_train.shape)

(3352, 8100)


In [6]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

In [7]:
pathtest = r'C:\AIstudy\traincarmodel\test'
dataset = readData(pathtest)

In [8]:
write_path = "imageHogTest.pkl"
pickle.dump(dataset, open(write_path,"wb"))

In [9]:
dataset = pickle.load(open('imageHogTest.pkl','rb'))

In [10]:
dataTest = np.array(dataset)
print(dataTest[1])

[0.04737655 0.06886563 0.09020519 ... 0.12020375 0.24305168 0.        ]


In [11]:
X_test = dataTest[:,0:-1]
y_test = dataTest[:,-1]
print(X_test.shape)

(813, 8100)


In [12]:
# %% Predict model DecisionTree
from sklearn import metrics

y_pred = clf.predict(X_test)
acc = metrics.accuracy_score(y_test,y_pred)
cofus_mect = metrics.confusion_matrix(y_test,y_pred)
print("accuracy = ",acc*100)
print('Confusion Mectrix :\n',cofus_mect)

accuracy =  46.61746617466175
Confusion Mectrix :
 [[ 71  25  17  25  17  24  20]
 [  9  40   3   3   2   3   7]
 [  9   3  25   9   6  11  12]
 [ 19   4   9  15   5   7  15]
 [  9   9   7   5  45   2  25]
 [ 12   3   3   8   4  66  10]
 [ 25  13   6  10  11   8 117]]


In [13]:
# %% Save model DecisionTree
path_model = 'modelTest.pkl'
pickle.dump(clf,open(path_model,'wb'))

In [14]:
# %% Train Model Xgboost
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train,y_train)

c:\AIstudy\traincarmodel\env\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)

In [15]:
# %% Predict model Xgboost
y_pred_xgb = xgb.predict(X_test)
acc_xgb = metrics.accuracy_score(y_test,y_pred_xgb)
cofus_mect_xgb = metrics.confusion_matrix(y_test,y_pred_xgb)
print("accuracy = ",acc_xgb*100)
print('Confusion Mectrix :\n',cofus_mect_xgb)

accuracy =  65.43665436654366
Confusion Mectrix :
 [[162   1   2   8   2   8  16]
 [  9  41   2   1   1   3  10]
 [  8   4  35   1   5   3  19]
 [ 41   1   4  15   4   0   9]
 [ 14   2   4   2  52   1  27]
 [ 16   1   4   0   3  76   6]
 [ 23   1   2   5   3   5 151]]


In [16]:
# %% Save model Xgboost
path_model = 'model_XGB.pk'
pickle.dump(xgb,open(path_model,'wb'))